In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds
import datetime
from Models.Transformer import Transformer
from Models.ESBNTransformer import ESBNTransformer2
from Utils.Logging import BatchLoggingModel
from Utils.preprocess_scan import generate_data
import tensorboard
tf.config.run_functions_eagerly(False)

/Users/jonathankonig/miniforge3/envs/esbn_abstractor/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Generate data and get vocab sizes
train_ds, val_ds, command_vocab_len, action_vocab_len = generate_data()

2024-04-02 11:08:14.060263: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


In [3]:
# Define evaluation metrics
def masked_loss(label, pred):
  mask = label != 0
  loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')
  loss = loss_object(label, pred)

  mask = tf.cast(mask, dtype=loss.dtype)
  loss *= mask

  loss = tf.reduce_sum(loss)/tf.reduce_sum(mask)
  return loss


def masked_accuracy(label, pred):
  pred = tf.argmax(pred, axis=2)
  label = tf.cast(label, pred.dtype)
  match = label == pred

  mask = label != 0

  match = match & mask

  match = tf.cast(match, dtype=tf.float32)
  mask = tf.cast(mask, dtype=tf.float32)
  return tf.reduce_sum(match)/tf.reduce_sum(mask)

In [4]:
num_layers = 2
d_model = 32
dff = 64
num_heads = 4
dropout_rate = 0.1

esbn_transformer = ESBNTransformer2(
    num_layers=num_layers,
    d_model=d_model,
    num_heads=num_heads,
    dff=dff,
    input_vocab_size=command_vocab_len,
    target_vocab_size=action_vocab_len,
    dropout_rate=dropout_rate)

In [5]:
esbn_transformer.compile(
    loss=masked_loss,
    optimizer="Adam",
    metrics=[masked_accuracy])

In [6]:
log_dir = "logs/fit/scan_transformer/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
esbn_transformer.fit(train_ds,
                epochs=20,
                validation_data=val_ds,
                callbacks=[tensorboard_callback])

Epoch 1/20
458/458 [==============================] - 35s 59ms/step - loss: 0.8096 - masked_accuracy: 0.6789 - val_loss: 2.9418 - val_masked_accuracy: 0.4670
Epoch 2/20
458/458 [==============================] - 28s 61ms/step - loss: 0.5072 - masked_accuracy: 0.7790 - val_loss: 2.9495 - val_masked_accuracy: 0.5098
Epoch 3/20
458/458 [==============================] - 28s 62ms/step - loss: 0.4231 - masked_accuracy: 0.8100 - val_loss: 3.2922 - val_masked_accuracy: 0.5317
Epoch 4/20
458/458 [==============================] - 29s 63ms/step - loss: 0.3400 - masked_accuracy: 0.8542 - val_loss: 3.3417 - val_masked_accuracy: 0.5387
Epoch 5/20
458/458 [==============================] - 29s 63ms/step - loss: 0.2230 - masked_accuracy: 0.9075 - val_loss: 3.2756 - val_masked_accuracy: 0.6271
Epoch 6/20
458/458 [==============================] - 29s 64ms/step - loss: 0.1680 - masked_accuracy: 0.9315 - val_loss: 3.3821 - val_masked_accuracy: 0.6215
Epoch 7/20
458/458 [==============================] 

KeyboardInterrupt: 

In [7]:
esbn_transformer.summary()

Model: "esbn_transformer2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 encoder (Encoder)           multiple                  42784     
                                                                 
 decoder (Decoder)           multiple                  76288     
                                                                 
 decoder_1 (Decoder)         multiple                  76288     
                                                                 
 esbn_encoder (ESBNEncoder)  multiple                  52323     
                                                                 
 dense_18 (Dense)            multiple                  650       
                                                                 
Total params: 248,333
Trainable params: 248,333
Non-trainable params: 0
_________________________________________________________________


In [ ]:
tf.config.run_functions_eagerly(True)

# Ordinary Transformer
num_layers = 2
d_model = 32
dff = 64
num_heads = 4
dropout_rate = 0.1

transformer = Transformer(
    num_layers=num_layers,
    d_model=d_model,
    num_heads=num_heads,
    dff=dff,
    input_vocab_size=len(command_processor.get_vocabulary()),
    target_vocab_size=len(action_processor.get_vocabulary()),
    dropout_rate=dropout_rate)

transformer.compile(
    loss=masked_loss,
    optimizer="Adam",
    metrics=[masked_accuracy])

log_dir = "logs/fit/scan_transformer/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
transformer.fit(train_ds,
                epochs=20,
                validation_data=val_ds,
                callbacks=[tensorboard_callback])

Epoch 1/20
458/458 [==============================] - 71s 154ms/step - loss: 0.8040 - masked_accuracy: 0.6832 - val_loss: 2.8252 - val_masked_accuracy: 0.4118
Epoch 2/20
458/458 [==============================] - 75s 163ms/step - loss: 0.4918 - masked_accuracy: 0.7875 - val_loss: 3.1255 - val_masked_accuracy: 0.4272
Epoch 3/20
458/458 [==============================] - 73s 160ms/step - loss: 0.4034 - masked_accuracy: 0.8237 - val_loss: 3.2123 - val_masked_accuracy: 0.4817
Epoch 4/20
458/458 [==============================] - 77s 169ms/step - loss: 0.3158 - masked_accuracy: 0.8664 - val_loss: 3.4009 - val_masked_accuracy: 0.5242
Epoch 5/20
458/458 [==============================] - 74s 161ms/step - loss: 0.2287 - masked_accuracy: 0.9048 - val_loss: 3.3856 - val_masked_accuracy: 0.5609
Epoch 6/20
458/458 [==============================] - 74s 162ms/step - loss: 0.1823 - masked_accuracy: 0.9242 - val_loss: 3.3009 - val_masked_accuracy: 0.5676
Epoch 7/20
458/458 [==========================

In [ ]:
transformer.summary()

Model: "transformer"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 encoder_1 (Encoder)         multiple                  85024     
                                                                 
 decoder_1 (Decoder)         multiple                  152256    
                                                                 
 dense_37 (Dense)            multiple                  330       
                                                                 
Total params: 237,610
Trainable params: 237,610
Non-trainable params: 0
_________________________________________________________________
